# Data Set を作ります。

In [ ]:
import numpy
import tensorflow
import pathlib
import sys
from PIL import Image

sys.path.append('..')
from src.utils import cut_data
from src.visualization import visualize

In [ ]:
# bad datas

# 一色丸々抜けてるシリーズ
bad_bub1 = [
    'S59'  , 'S77'  , 'CS89' ,
]

# 24um で大幅に中心がぬけているシリーズ
bad_bub2 = [
    'CN109', 'CN113', 'CN120', 'CN135', 'CN145',
    'CN148', 'N10'  , 'N39'  , 'N52'  , 'N101' ,
    'CS81' , 'CS82' , 'CS83' , 'CS84' , 'CN8'  ,
    'CS102', 'CS103', 'CS112', 'S6'   , 'CS90' ,
    'S144' , 'S149' , 'S155' , 'S181' , 'CN7'  ,
    'CN14' , 'CN16' , 'CS8'  , 'CS13' , 'CN78' ,
    'CN102', 'CS55' , 'CS70' , 'S153' , 'S48'  ,
    'S51'  ,
]

# 色やばいやつ
bad_bub3 = [
    'CN143', 'CS85' , 'CS86' , 'CS88' , 'S154' ,
]

# 端にあることでリング部のデータないやつら
bad_bub4 = [
    'N5'   , 'S12'  , 'S61'  , 'S74'  , 'S185' ,
]

# サイズが大きい + データの端に位置しているせいで、中心がずれたと思われるやつら
bad_bub5 = [
    'N63'  , 'N109' ,
]

bad_bub = bad_bub1 + bad_bub2 + bad_bub3 + bad_bub4 + bad_bub5

In [ ]:
# DataSet作成用
sdf = cut_data.get_spitzer_df(
    path = '~/jupyter/spitzer_bubble/data/interim/gal',
    fac  = 0.1,            # Ring に対して non-Ring を何倍生成するか
    b    = [-0.8, 0.8],   # non-Ring の b の中心座標範囲 (広くしすぎるとはみ出る可能性があがる)
    R    = [0.1, 20],     # Rout の範囲を決める
    seed = 1              # Seed 値 (指定すると、毎回同じ non-Ring データを生成できる)
)

sdf.drop_obj(bad_bub)
sdf.df

# dir_ = sdf.get_dir()
# dir_

In [ ]:
# 生データの pd を クロスバリデーション用に分割
from sklearn.model_selection import StratifiedKFold

fold_num = 5
seed = 7
numpy.random.seed(seed)
kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)

X = sdf.df.copy()
Y = numpy.array(sdf.df['label'].tolist())

x_train_raw_list = []
x_test_raw_list = []
y_train_raw_list = []
y_test_raw_list = []

for train_index, test_index in kfold.split(X,Y):
    # 新たに index を追加 + 古い index を削除も同時に行う
    x_train_raw_list.append(X.iloc[train_index])#.reset_index())
    x_test_raw_list.append(X.iloc[test_index])#.reset_index())
    y_train_raw_list.append(Y[train_index])
    y_test_raw_list.append(Y[test_index])

In [ ]:
train = x_train_raw_list[0]
test = x_test_raw_list[0]

In [ ]:
sdf.df = train

datas = []
infos = []

for dir_ in sdf.get_dir():
    cut_tb = sdf.get_cut_table(dir_, 1.5)
    for obj in cut_tb.get_obj():
        data, info = cut_tb[obj]
        
        data = tensorflow.image.resize(
            images=data, 
            size=[64, 64], 
            method='bilinear',
        )
        
        datas.append(data)
        infos.append(info)

x_train = numpy.concatenate(datas)
train_infos = infos
y_train = numpy.array([info['label'] for info in train_infos])

In [ ]:
sdf.df = test

datas = []
infos = []

for dir_ in sdf.get_dir():
    cut_tb = sdf.get_cut_table(dir_, 1.5)
    for obj in cut_tb.get_obj():
        data, info = cut_tb[obj]
        
        data = tensorflow.image.resize(
            images=data, 
            size=[64, 64], 
            method='bilinear',
        )
        
        datas.append(data)
        infos.append(info)

x_test = numpy.concatenate(datas)
test_infos = infos
y_test = numpy.array([info['label'] for info in test_infos])

In [ ]:
# img = tensorflow.image.resize(
#     images=numpy.array([data]), 
#     size=[512, 512], 
#     method='gaussian',
# )
# img = numpy.where(img>255, 255, img)
# im = Image.fromarray(numpy.uint8(img[0]))
# im

In [ ]:
# img = tensorflow.image.resize(
#     images=numpy.array([data]), 
#     size=[512, 512], 
#     method='gaussian',
# )
# img = numpy.where(img>255, 255, img)
# img = tensorflow.image.random_crop(img[0], [400, 400, 3])
# im = Image.fromarray(numpy.uint8(img))
# im

In [ ]:
# def znormalize(data, each_color=True):
#     f = lambda x: ((x-x.mean())/numpy.std(x)+1)*(255/2) 
    
#     if each_color:
#         rgb = [data[:, :, i] for i in range(data.shape[2])]
#         data = numpy.stack([f(data) for data in rgb], axis=2)
#         pass    
    
#     if not each_color:
#         data = f(data)
#         pass
    
#     return data

In [ ]:
# mkdir の機能を加える必要あり
save_dir = ''

save_path = pathlib.Path('../data/processed/dataset')
numpy.save(save_path/save_dir/'x_train.npy', x_train)
numpy.save(save_path/save_dir/'y_train.npy', y_train)
numpy.save(save_path/save_dir/'x_test.npy', x_test)
numpy.save(save_path/save_dir/'y_test.npy', y_test)

# Data を見たりしたい

In [ ]:
# 良いRingデータのみ抜き出すやつ
sdf = cut_data.get_spitzer_df(
    path = '~/jupyter/spitzer_bubble/data/interim/gal',
    fac  = 1,             # Ring に対して non-Ring を何倍生成するか
    b    = [-0.8, 0.8],   # non-Ring の b の中心座標範囲 (広くしすぎるとはみ出る可能性があがる)
    R    = [0.1, 20],     # Rout の範囲を決める
    seed = 1              # Seed 値 (指定すると、毎回同じ non-Ring データを生成できる)
)

sdf.drop_obj(bad_bub)
sdf.drop_label(0)
sdf.df

# dir_ = sdf.get_dir()
# dir_

In [ ]:
# 悪いRingデータのみ抜き出すやつ
sdf = cut_data.get_spitzer_df(
    path = '~/jupyter/spitzer_bubble/data/interim/gal',
    fac  = 1,             # Ring に対して non-Ring を何倍生成するか
    b    = [-0.8, 0.8],   # non-Ring の b の中心座標範囲 (広くしすぎるとはみ出る可能性があがる)
    R    = [0.1, 20],     # Rout の範囲を決める
    seed = 1              # Seed 値 (指定すると、毎回同じ non-Ring データを生成できる)
)

sdf.drop_label(0)
sdf.select_obj(bad_data)

# dir_ = sdf.get_dir()
# dir_

In [ ]:
# Rout 10 ~ 20 の間のやつ
sdf = cut_data.get_spitzer_df(
    path = '~/jupyter/spitzer_bubble/data/interim/gal',
    fac  = 1,             # Ring に対して non-Ring を何倍生成するか
    b    = [-0.8, 0.8],   # non-Ring の b の中心座標範囲 (広くしすぎるとはみ出る可能性があがる)
    R    = [0.1, 20],     # Rout の範囲を決める
    seed = 1              # Seed 値 (指定すると、毎回同じ non-Ring データを生成できる)
)

sdf.drop_label(0)
sdf.select_obj(['N69', 'N109', 'S13', 'CS61', 'N63', 'N81', 'N107'])

# dir_ = sdf.get_dir()
# dir_

In [ ]:
sdf.df

In [ ]:
datas = []
infos = []
for dir_ in sdf.get_dir():
    cut_tb = sdf.get_cut_table(dir_)
    for obj in cut_tb.get_obj():
        data, info = cut_tb[obj]
        #data = znormalize(data)
        data = tensorflow.image.resize(
            images=data, 
            size=[512, 512], 
            method='gaussian',
        )

        datas.append(data)
        infos.append(info)
        
datas = numpy.concatenate(datas)
datas = numpy.where(datas>255, 255, datas)

In [ ]:
imgs = tensorflow.image.resize(
    images=datas, 
    size=[128, 128], 
    method='gaussian',
)

In [ ]:
visualize.data_view(20, imgs, infos)

## 以下、試行錯誤の結果

## get_sample_table(汚い関数)で Data Set のテーブルを取得 & RGB FITS への Path 取得

In [ ]:
sample = data_utils.get_sample_table()
data_path = pathlib.Path('~/jupyter/spitzer_bubble/data/interim/gal').expanduser()

## 天体名を指定して RGB の HDU を取得 & data 取り出し

In [ ]:
obj = 'S10'
dir_ = sample.loc[:, 'directory'][obj]
files =['r.fits', 'g.fits', 'b.fits']
hdus = [astropy.io.fits.open(data_path/dir_/file)[0] for file in files]

r_data = hdus[0].data
g_data = hdus[1].data
b_data = hdus[2].data

# r_data = numpy.where(r_data>255, 255, r_data)
# r_data = numpy.where(r_data<0, 0, r_data)
# g_data = numpy.where(g_data>255, 255, g_data)
# g_data = numpy.where(g_data<0, 0, g_data)
# b_data = numpy.where(b_data>255, 255, b_data)
# b_data = numpy.where(b_data<0, 0, b_data)

#rgb_data = numpy.stack([r_data, g_data, b_data], 2)

## header と天体の座標から nparray の pixcel 範囲を計算

In [ ]:
margin = 3

header = hdus[0].header
w = astropy.wcs.WCS(header)

l_min = sample.loc[obj]['l'] - margin*sample.loc[obj]['Rout']/60
b_min = sample.loc[obj]['b'] - margin*sample.loc[obj]['Rout']/60
l_max = sample.loc[obj]['l'] + margin*sample.loc[obj]['Rout']/60
b_max = sample.loc[obj]['b'] + margin*sample.loc[obj]['Rout']/60
x_pix_min, y_pix_min = w.all_world2pix(l_max, b_min, 0)
x_pix_max, y_pix_max = w.all_world2pix(l_min, b_max, 0)

R_pix = int(((x_pix_max - x_pix_min)/2 + (y_pix_max - y_pix_min)/2)/2)
x_pix, y_pix = w.all_world2pix(sample.loc[obj]['l'], sample.loc[obj]['b'], 0)

x_pix_min = int(numpy.round(x_pix)) - R_pix
x_pix_max = int(numpy.round(x_pix)) + R_pix
y_pix_min = int(numpy.round(y_pix)) - R_pix
y_pix_max = int(numpy.round(y_pix)) + R_pix

if x_pix_min < 0:
    x_pix_min = 0
    pass
if y_pix_min < 0:
    y_pix_min = 0
    pass

In [ ]:
r_obj = r_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
g_obj = g_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]
b_obj = b_data[y_pix_min:y_pix_max, x_pix_min:x_pix_max]

rgb_data = numpy.stack([r_obj, g_obj, b_obj], 2)
rgb_data = numpy.flipud(rgb_data)

In [ ]:
rgb_data = tensorflow.image.resize(
    images=numpy.array([rgb_data]),
    size=[256, 256],
    method='bilinear',
    preserve_aspect_ratio=True,
)
im = Image.fromarray(numpy.uint8(rgb_data[0]))
im

In [ ]:
# 2020/03/27 追記
# cut_data の中にデータを正方化するメソッドを追加

def padding_obj(data, info):
    pad = data.shape[0] - data.shape[1]
    if pad > 0 and info['x_pix_min'] != 0:
        data = numpy.pad(data, [(0, 0),(0, pad), (0, 0)])
        pass
    
    if pad > 0 and info['x_pix_min'] == 0:
        data = numpy.pad(data, [(0, 0),(pad, 0), (0, 0)])
        pass

    if pad < 0 and info['y_pix_min'] == 0:
        data = numpy.pad(data, [(0, abs(pad)),(0, 0), (0, 0)])
        pass
    
    if pad < 0 and info['y_pix_min'] != 0:
        data = numpy.pad(data, [(abs(pad), 0),(0, 0), (0, 0)])
        pass
    
    return data